# Notebook 10: Frequency-Domain Specifications and Loop Shaping
© 2024 ETH Zurich, Mark Benazet Castells, Jonas Holinger, Felix Muller, Matteo Penlington; Institute for Dynamic Systems and Control; Prof. Emilio Frazzoli

This interactive notebook explores frequency-domain specifications and loop shaping approaches for control system design, covering concepts like gain and phase margins, performance limitations, and systematic control synthesis methods.

Authors:
- Felix Muller; fmuller@ethz.ch
- Mark Benazet Castells; mbenazet@ethz.ch

## Learning Objectives

After completing this material, you should be able to:

1. Understand and specify control requirements in the frequency domain:
   - Express command tracking requirements 
   - Specify disturbance rejection requirements
   - Define noise rejection specifications

2. Use loop shaping techniques for control design:
   - Apply proportional control effectively
   - Design lead compensation for phase margin improvement
   - Design lag compensation for steady-state error reduction
   - Combine compensation elements systematically

3. Understand fundamental performance limitations:
   - Recognize the waterbed effect
   - Handle constraints from non-minimum phase zeros
   - Deal with limitations from unstable poles
   - Apply Bode's integral theorem

## Required Packages

Run the following cell to import required packages:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interactive_output, FloatSlider, HBox, VBox
from IPython.display import display, clear_output, HTML
from scipy import signal
import control

## 1. Frequency-Domain Specifications

### 1.1 Introduction 

Frequency-domain specifications provide a powerful way to express control requirements. The key idea is to separate different control objectives by frequency:

- Low frequencies: Command tracking and disturbance rejection
- High frequencies: Noise rejection 
- Mid frequencies: Stability margins and robustness

For a closed-loop system, we work with two key transfer functions:

1. Sensitivity function: $S(s) = \frac{1}{1 + L(s)}$
   - $S(s)$ captures the system's ability to reject disturbances and determines error in tracking commands
2. Complementary sensitivity function: $T(s) = \frac{L(s)}{1 + L(s)}$
   - $T(s)$ captures the system's ability to track commands and determines the effect of noise.

where $L(s) = P(s)C(s)$ is the loop transfer function.

Key specifications are typically written as:

1. Command tracking/disturbance rejection:
   - $|S(j\omega)| \ll 1$ at low frequencies
   - Example: "ensure commands are tracked with max 10% error up to 10Hz"

2. Noise rejection:
   - $|T(j\omega)| \ll 1$ at high frequencies  
   - Example: "ensure noise is reduced by factor of 10 at frequencies above 100Hz"

### 1.3 Translating to Open-Loop Specifications
The closed-loop specifications can be translated to requirements on the open-loop transfer function $L(s)$:

1. Command tracking/disturbance rejection requirement:
   - $|S(j\omega)| \cdot |W_1(j\omega)| < 1$ 
   - This translates to $|L(j\omega)| > |W_1(j\omega)|$

2. Noise rejection requirement:
   - $|T(j\omega)| \cdot |W_2(j\omega)| < 1$
   - This translates to $|L(j\omega)| < |W_2(j\omega)|^{-1}$

3. Cross-over frequency:
   - The frequency where $|L(j\omega_{gc})| = 1$ is called the cross-over frequency $\omega_{gc}$

Where $W_1(j\omega)$ and $W_2(j\omega)$ are weighting functions that capture the desired behavior at low and high frequencies, respectively.

These create an "obstacle course" on the Bode plot through which we must navigate $L(j\omega)$. The goal would be to find a loop transfer function that stays between the two curves. 

<div style="text-align:center;">
<img src="./img/obstacle.png" alt="Obstacle" width="500">
</div>

The challenge is to design L(jω)L(j\omega)L(jω) such that:

   - It stays above $W1W_1W1$​ at low frequencies
   - It stays below W2−1W_2^{-1}W2−1​ at high frequencies
   - It transitions smoothly between these bounds near crossover

# 2. Loop Shaping

We've seen how different control specifications create a sort of "obstacle course" in the frequency domain. This can feel quite abstract at first - how do we actually shape our loop transfer function L(s) to navigate through these obstacles? 

Let's explore this interactively! Below is a tool that lets you experiment with the fundamental building blocks of loop shaping:

1. **Proportional Gain (k)**:
   - The simplest tool in our toolkit 
   - Shifts the magnitude plot up/down uniformly
   - No effect on phase
   - What happens to stability as we increase gain?

2. **Lead Compensation** ($\frac{s/a + 1}{s/b + 1}, 0 < a < b$):
   - Our tool for boosting phase margin
   - Maximum phase boost occurs around frequency $\sqrt{ab}$
   - Try placing this near your desired crossover frequency!

3. **Lag Compensation** ($\frac{s/a + 1}{s/b + 1}, 0 < b < a$):
   - Great for improving low-frequency performance
   - Needs to be placed well below crossover to avoid phase loss
   - Can you use this to meet the tracking specifications?

The plot shows:
- Plant response in green (try changing it with the numerator/denominator fields!)
- Compensated system in blue 
- Red region: Must stay above this at low frequencies for good tracking
- Yellow region: Must stay below this at high frequencies to reject noise

Play around with different combinations - what works? What doesn't? Can you find a simple solution that meets all specifications? There's often more than one way to succeed!

In [ ]:
def create_plant(num_str, den_str):
    """Create plant transfer function from string inputs"""
    try:
        num = [float(n) for n in num_str.split(',')]
        den = [float(d) for d in den_str.split(',')]
        return control.TransferFunction(num, den)
    except:
        # Return default plant if there's an error
        return control.TransferFunction([0.01, 0.2, 2], [1, 2, 2, 0])

def create_compensator(K, lead_zero, lead_pole, lag_zero, lag_pole, use_lead=True, use_lag=True):
    """Create compensator transfer function based on parameters"""
    comp = control.TransferFunction([K], [1])  # Start with gain
    
    if use_lead:
        # Lead compensator: (s/a + 1)/(s/b + 1) where b > a
        lead_num = [1/lead_zero, 1]
        lead_den = [1/lead_pole, 1]
        comp = comp * control.TransferFunction(lead_num, lead_den)
    
    if use_lag:
        # Lag compensator: (s/a + 1)/(s/b + 1) where a > b
        lag_num = [1/lag_zero, 1]
        lag_den = [1/lag_pole, 1]
        comp = comp * control.TransferFunction(lag_num, lag_den)
    
    return comp

def plot_bode(K=1, lead_zero=0.3, lead_pole=30, lag_zero=10, lag_pole=0.1, 
              use_lead=True, use_lag=True, show_bounds=True,
              num_str='0.01,0.2,2', den_str='1,2,2,0'):
    """Plot Bode diagram with compensator effects"""
    plt.figure(figsize=(12, 8))
    
    # Create plant and compensator
    plant = create_plant(num_str, den_str)
    comp = create_compensator(K, lead_zero, lead_pole, lag_zero, lag_pole, use_lead, use_lag)
    
    # Get loop transfer function
    L = plant * comp
    
    # Generate frequency points
    w = np.logspace(-2, 3, 1000)
    
    # Get frequency response
    mag, phase, w = control.bode(L, w, plot=False)
    
    # Plot magnitude
    plt.subplot(211)

    # Get and plot plant frequency response
    mag_p, phase_p, _ = control.bode(plant, w, plot=False)
    plt.subplot(211)
    plt.semilogx(w, 20 * np.log10(mag_p), 'g-', label='Plant', alpha=0.7)

    plt.subplot(212)
    plt.semilogx(w, phase_p * 180/np.pi, 'g-', alpha=0.7)

    # Then plot L(s) in blue
    plt.subplot(211)
    plt.semilogx(w, 20 * np.log10(mag), 'b-', label='Compensated system')
    
    # Add performance bounds
    if show_bounds:
        # W1 bound (low-frequency box for disturbance rejection)
        # Must stay ABOVE this at low frequencies
        w1_x = [w[0], 0.1, 0.1, w[0]]
        w1_y = [20, 20, -100, -100]
        plt.fill(w1_x, w1_y, alpha=0.5, color='r', label='Load disturbance\nattenuation')

        # W2 bound (high-frequency box for noise rejection)
        # Must stay BELOW this at high frequencies
        w2_x = [100, w[-1], w[-1], 100]
        w2_y = [100, 100, -25, -25]
        plt.fill(w2_x, w2_y, alpha=0.5, color='y', label='High frequency\nmeasurement noise')

    plt.grid(True)
    plt.ylabel('Magnitude [dB]')
    plt.legend()
    
    # Add reference lines
    plt.axhline(y=0, color='k', linestyle=':')
    
    # Plot phase
    plt.subplot(212)
    plt.semilogx(w, phase * 180/np.pi, 'b-')
    plt.grid(True)
    plt.ylabel('Phase [deg]')
    plt.xlabel('Frequency [rad/s]')
    
    # Add phase margin indicators
    gain_crossover_idx = np.abs(mag - 1).argmin()
    if np.abs(mag[gain_crossover_idx] - 1) < 0.1:  # Only show if we actually cross 0dB
        gain_crossover = w[gain_crossover_idx]
        phase_at_crossover = phase[gain_crossover_idx] * 180/np.pi
        phase_margin = 180 + phase_at_crossover
        
        plt.subplot(211)
        plt.axvline(gain_crossover, color='r', linestyle='--', alpha=0.3)
        
        plt.subplot(212)
        plt.axvline(gain_crossover, color='r', linestyle='--', alpha=0.3)
        plt.plot(gain_crossover, phase_at_crossover, 'ro')
        plt.text(gain_crossover, phase_at_crossover, f'PM = {phase_margin:.1f}°')
        
        # Add reference lines
        plt.axhline(y=-180, color='k', linestyle=':')
    
    plt.tight_layout()

# Create interactive widgets
numerator_input = widgets.Text(
    value='0.01,0.2,2',
    description='Numerator:',
    style={'description_width': 'initial'}
)

denominator_input = widgets.Text(
    value='1,2,2,0',
    description='Denominator:',
    style={'description_width': 'initial'}
)

K_slider = FloatSlider(value=1, min=0.1, max=1000, step=0.1, description='Gain K')
lead_zero_slider = FloatSlider(value=0.3, min=0.01, max=10, step=0.1, description='Lead Zero')
lead_pole_slider = FloatSlider(value=30, min=1, max=100, step=1, description='Lead Pole')
lag_zero_slider = FloatSlider(value=10, min=0.1, max=50, step=0.1, description='Lag Zero')
lag_pole_slider = FloatSlider(value=0.1, min=0.01, max=1, step=0.01, description='Lag Pole')
use_lead = widgets.Checkbox(value=True, description='Use Lead')
use_lag = widgets.Checkbox(value=True, description='Use Lag')
show_bounds = widgets.Checkbox(value=True, description='Show Bounds')

# Create interactive plot
interactive_plot = interactive_output(plot_bode, 
                                   {'K': K_slider,
                                    'lead_zero': lead_zero_slider,
                                    'lead_pole': lead_pole_slider,
                                    'lag_zero': lag_zero_slider,
                                    'lag_pole': lag_pole_slider,
                                    'use_lead': use_lead,
                                    'use_lag': use_lag,
                                    'show_bounds': show_bounds,
                                    'num_str': numerator_input,
                                    'den_str': denominator_input})

# Display widgets and plot
display(VBox([
    HBox([numerator_input, denominator_input]),
    HBox([K_slider]),
    HBox([use_lead, lead_zero_slider, lead_pole_slider]),
    HBox([use_lag, lag_zero_slider, lag_pole_slider]),
    HBox([show_bounds]),
    interactive_plot
]))